## "Detection of Anomalies in Financial Transactions using Deep Autoencoder Networks"

This GPU Technology Conference (GTC) 2018 lab was developed by Mr. X, and Mr. Y

## 01. Environment Verification

#### 01.1 Python Verification

Before we begin, let's verify that Python is working on your system. To do this, execute the cell block below by giving it focus (clicking on it with your mouse), and hitting Shift-Enter, or pressing the play button in the toolbar above. If all goes well, you should see some output returned below the grey cell.

In [4]:
print("The answer should be forty-two: " + str(40+2))

The answer should be forty-two: 42


#### 01.2 GPU Verficiation

Let's execute the cell below to display information about the GPUs running on the server.

In [5]:
!nvidia-smi

/bin/sh: nvidia-smi: command not found
